In [14]:
%load_ext autoreload
%autoreload 2
import h5py, os
import numpy as np
import tensorflow as tf
import layers
import metrics_for_loading
import basenji_model_for_model_zoo
tf.compat.v1.disable_eager_execution()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
input_size = 1024
label_num = 7
TFR_INPUT = 'sequence'
TFR_OUTPUT = 'target'

In [22]:
def parse_proto(example_protos):
    """Parse TFRecord protobuf."""

    # define features
    features = {
    TFR_INPUT: tf.io.FixedLenFeature([], tf.string),
    TFR_OUTPUT: tf.io.FixedLenFeature([], tf.string)
    }

    # parse example into features
    parsed_features = tf.io.parse_single_example(example_protos, features=features)

    # decode sequence
    sequence = tf.io.decode_raw(parsed_features[TFR_INPUT], tf.uint8)
    sequence = tf.reshape(sequence, [1024, 4])
    sequence = tf.cast(sequence, tf.float32)

    # decode targets
    targets = tf.io.decode_raw(parsed_features[TFR_OUTPUT], tf.float16)
    # if not raw:
    targets = tf.reshape(targets, [32, 7])
    targets = tf.cast(targets, tf.float32)

    return sequence, targets

def create_model(input_size, label_num):
    model = basenji_model_for_model_zoo.basenji_small((input_size, 4), label_num)
    model_metrics = [metrics_for_loading.PearsonR(label_num), metrics_for_loading.R2(label_num)]
    model.compile(loss=tf.keras.losses.Poisson(),
              optimizer='sgd',
              metrics=model_metrics)
    return model

In [24]:
model = create_model(input_size, label_num)

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sequence (InputLayer)           [(None, 1024, 4)]    0                                            
__________________________________________________________________________________________________
stochastic_reverse_complement_3 ((None, 1024, 4), () 0           sequence[0][0]                   
__________________________________________________________________________________________________
stochastic_shift_3 (StochasticS (None, 1024, 4)      0           stochastic_reverse_complement_3[0
__________________________________________________________________________________________________
gelu_21 (GELU)                  (None, 1024, 4)      0           stochastic_shift_3[0][0]         
____________________________________________________________________________________________

In [26]:
model.load_weights('/home/shush/profile/basenji/models/HepG2_sign/model.h5')

In [26]:
# reloaded_model = tf.keras.experimental.load_from_saved_model('../models/HepG2_sign/model.h5', 
#                                            custom_objects=custom_objects=
#                                           {'StochasticReverseComplement': layers.StochasticReverseComplement,
#                                           'StochasticShift': layers.StochasticShift,
#                                           'GELU': layers.GELU,
#                                           'SwitchReverse': layers.SwitchReverse,
#                                           'PearsonR': metrics_for_loading.PearsonR,
#                                           'R2': metrics_for_loading.R2
#                                           })
# print(reloaded_model.get_config())

# #Get input shape from model.get_config()
# reloaded_model.build((None, 224, 224, 3))
# reloaded_model.summary()

In [27]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sequence (InputLayer)           [(None, 1024, 4)]    0                                            
__________________________________________________________________________________________________
stochastic_reverse_complement_3 ((None, 1024, 4), () 0           sequence[0][0]                   
__________________________________________________________________________________________________
stochastic_shift_3 (StochasticS (None, 1024, 4)      0           stochastic_reverse_complement_3[0
__________________________________________________________________________________________________
gelu_21 (GELU)                  (None, 1024, 4)      0           stochastic_shift_3[0][0]         
____________________________________________________________________________________________